In [1]:
import os

In [2]:
%pwd

'/Users/jharna/work/self_learning/MLOps/kidney-disease-classification-deeplearning-pytorch/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/jharna/work/self_learning/MLOps/kidney-disease-classification-deeplearning-pytorch'

In [5]:
# define entity (dataclass)
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifierKidneyDisease.constants import *
from cnnClassifierKidneyDisease.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(self,
    config_file_path = CONFIG_FILE_PATH,
    params_file_path = PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [19]:
import zipfile
from cnnClassifierKidneyDisease.utils.common import get_size
from cnnClassifierKidneyDisease import logger
import gdown

In [20]:
class DataIngestion():
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Fetch data from the url
        """
        try:
            print(self.config)
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True)
            logger.info(f"Downloading from {dataset_url} into {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix_url = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix_url+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        """
        zip file path is a str
        extracts zip file and saves to data directory
        function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-29 23:05:58,614: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-05-29 23:05:58,615: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-29 23:05:58,616: INFO: common: created directory at: artifacts]
[2024-05-29 23:05:58,617: INFO: common: created directory at: artifacts/data_ingestion]
DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://drive.google.com/file/d/1ndhaI6Z8H4VKZPo4bml2Ml6XyoR9DcdL/view?usp=sharing', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')
[2024-05-29 23:05:58,617: INFO: 1956852303: Downloading from https://drive.google.com/file/d/1ndhaI6Z8H4VKZPo4bml2Ml6XyoR9DcdL/view?usp=sharing into artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1ndhaI6Z8H4VKZPo4bml2Ml6XyoR9DcdL
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1ndhaI6Z8H4VKZPo4bml2Ml6XyoR9DcdL&confirm=t&uuid=7e1c3c02-5301-429f-8b7a-fdc328a35052
To: /Users/jharna/work/self_learning/MLOps/kidney-disease-classification-deeplearning-pytorch/artifacts/data_ingestion/data.zip
100%|██████████| 943M/943M [00:18<00:00, 51.5MB/s] 

[2024-05-29 23:06:18,146: INFO: 1956852303: Downloaded data from https://drive.google.com/file/d/1ndhaI6Z8H4VKZPo4bml2Ml6XyoR9DcdL/view?usp=sharing into file artifacts/data_ingestion/data.zip]
